In [7]:
import numpyro
# numpyro.util.set_host_device_count(4)
# import numpyro.distributions as dist
# from numpyro.infer import MCMC, NUTS, Predictive

# from jax import numpy as jnp
# from jax import random

import numpy as np
import pandas as pd
import seaborn as sns
import arviz as az

import siuba as s
from siuba import _
from plotnine import *
%run -i 'model_helpers.py'

In [16]:
df2 = load_data_exp1() # see data_helpers.py
df2.head()

,ID,block,trial,query,querydetail,querytype,estimate,starttime,endtime,RT,condition
0,0,1,1,What is the probability that the weather will ...,not frosty,notB,0.6,1.122868e+06,1.122878e+06,9.729185,0
1,0,1,2,If the weather in England is normal on a rando...,typical given normal,BgA,0.5,1.122878e+06,1.122893e+06,14.901229,0
2,0,1,3,What is the probability that the weather will ...,normal,A,0.5,1.122893e+06,1.122903e+06,9.686761,0
3,0,1,4,What is the probability that the weather will ...,not icy or not frosty,notAornotB,0.5,1.122903e+06,1.122911e+06,7.788618,0
4,0,1,5,If the weather in England is normal on a rando...,not typical given normal,notBgA,0.7,1.122911e+06,1.122920e+06,8.626205,0


In [17]:
(
df2 >>
    s.group_by(_.ID, _.querydetail) >>
    s.mutate(
        avg_est = np.mean(_.estimate),
        sd_est = np.std(_.estimate)
    ) >>
    s.mutate(est_diff = _.estimate - _.avg_est) >>
    s.ungroup() >>
    s.summarize(x = np.mean(_.est_diff.transform(np.abs) > .3))
    # ggplot(aes(x = "est_diff")) +
    # geom_histogram()
)
## in exp2, 3.5% differ from avg by more than .30 -- those are big outliers
## and that much contamination will ruin estimation based on my simulations.
## in exp1, it's 7% !

,x
0,0.071045


At first I thought the use of 3 measurement blocks was very strange. But now I see it is solving for a response problem: a non-negligible percent of the time participants seem to just guess at a number. These contaminants are a big problem for estimation, but they get smoothed over if you average multiple trials together. Presumably those trials need to be fairly separated too, so that people don't just guess repeatedly on the same trials. 

Based on the numbers, I'd wager 5% or more of trials are guesses or contaminants. But the chance you just guess more than once on the same trial are then quite low (e.g. ~.25%) so the trial means are still useful.

To solve for this I will either need to get a real working model of the contaminants sampling well, or I will need to adjust my data collection method to be more similar to Zhu et al.